In [1]:
import numpy, pandas
import os

In [2]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
import random
from functools import reduce

import json, os
import traceback
import matplotlib.pyplot as plt
%matplotlib inline

import wget

In [3]:
from groupify.app.spotify_utils import *
from groupify.app.ml_utils import *

In [4]:
redirect_uri = 'https://example.com'

In [54]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import groupify.app.psecrets as psecrets

try: 
    cid = psecrets.client_id
    secret = psecrets.secret
    userdata = json.load(open('groupify/app/json/userdata.json', 'r'))
    username = list(userdata.keys())[0]
    scope = 'user-top-read user-read-recently-played user-follow-read playlist-modify-public user-library-read playlist-read-collaborative'
    token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
except: 
    print(traceback.format_exc(5))


In [6]:
list_of_genres = pandas.read_html('https://everynoise.com/everynoise1d.cgi?scope=all', index_col=0)
list_of_genres = list_of_genres[0].replace('☊', '')
list_of_genres = list_of_genres.values.tolist()
list_of_genres = [a[1] for a in list_of_genres]
list_of_genres

['pop',
 'dance pop',
 'rap',
 'rock',
 'hip hop',
 'latin',
 'pop rap',
 'trap latino',
 'trap',
 'modern rock',
 'edm',
 'reggaeton',
 'pop dance',
 'post-teen pop',
 'latin pop',
 'electropop',
 'pop rock',
 'classic rock',
 'tropical house',
 'r&b',
 'mellow gold',
 'musica mexicana',
 'soft rock',
 'permanent wave',
 'album rock',
 'melodic rap',
 'southern hip hop',
 'alternative metal',
 'k-pop',
 'urban contemporary',
 'canadian pop',
 'uk pop',
 'electro house',
 'contemporary country',
 'indie pop',
 'alternative rock',
 'alt z',
 'gangster rap',
 'nueva musica mexicana',
 'hard rock',
 'nu metal',
 'country',
 'ranchera',
 'rock en espanol',
 'tropical',
 'latin hip hop',
 'desi pop',
 'art pop',
 'neo mellow',
 'underground hip hop',
 'adult standards',
 'atl hip hop',
 'modern alternative rock',
 'french hip hop',
 'country road',
 'folk rock',
 'arrocha',
 'chicago rap',
 'filmi',
 'new wave pop',
 'pop urbaine',
 'german hip hop',
 'art rock',
 'soul',
 'country rock',
 

In [7]:
num_playlists=3
num_genres = 10
genres_per_playlist = 3

simulated_playlists = []

for ind in range(num_playlists): 
    pl_indexes = numpy.random.choice(num_genres, genres_per_playlist, replace=False) # 3 genres per playlist
    genres = [list_of_genres[a] for a in pl_indexes]
    print('Generating playlists for these genres: ', genres)
    trackids = get_recommendations_tracks_from_genre(sp, genres, limit=10)
    print(len(trackids))
    simulated_playlists.append(trackids)
# print([len(a) for a in simulated_playlists])

Generating playlists for these genres:  ['hip hop', 'latin', 'trap']
93
Generating playlists for these genres:  ['hip hop', 'dance pop', 'pop']
109
Generating playlists for these genres:  ['dance pop', 'modern rock', 'pop']
142


## Goal: 
Given a playlist parameterized by list of track_ids, we want to 
1) generate a feature matrix of dimension num_tracks_in_playlist x num_features
2) Perform unsupervised learning to find a latent representation that maximizes the dimensions that are common to both playlists --- baseline PCA, ideal: autoencoders. 

## Function definitions for features extraction 

In [8]:
def get_genre_from_track(sp, trackid, limit=1): 
    track = sp.track(trackid)
    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    genres = artist["genres"]
    if len(genres): 
        return genres[0:limit]
    else: 
        return []

In [9]:
def get_audio_df(sp, trackids, numerical_features, categorical_features): 

    feature_df = pandas.DataFrame(sp.audio_features(trackids))
    feature_df = feature_df[numerical_features + categorical_features ]
    feature_df[categorical_features] = feature_df[categorical_features].astype(str)
    feature_df = pandas.get_dummies(feature_df, drop_first = True, columns = categorical_features)
    feature_df['id'] = trackids
    return feature_df

In [10]:
def get_stats(arr, col_prefix='', pvalues = [0, 25, 50, 75,100]): 
    assert isinstance(arr, numpy.ndarray) or isinstance(arr, list)
    if isinstance(arr, list): 
        arr = numpy.array(arr)
    percentiles = numpy.round(numpy.percentile(arr, pvalues, axis = 0), decimals = 2).tolist()
    pvalues = [col_prefix + '_' + str(p) for p in pvalues]
    d = dict(zip(pvalues, percentiles))
    return d

In [11]:
get_stats(numpy.random.rand(5), 'test')

{'test_0': 0.13,
 'test_25': 0.15,
 'test_50': 0.53,
 'test_75': 0.63,
 'test_100': 0.84}

In [12]:
def get_mean_variance(arr, col_prefix=''): 
    d = {}
    d[col_prefix + '_mean'] = numpy.round(numpy.mean(arr, axis = None), 2)
    d[col_prefix + '_std'] = numpy.round(numpy.std(arr, axis = None), 2)
    return d

In [81]:
def gen_basic_audio_features(sp, sample_trackid): 
    numerical_features= ['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms']
    categorical_features = ['key', 'mode']
    audio_feats = sp.audio_features(sample_trackid)[0]
    feature = {}
    for feat in numerical_features: 
        feature[feat] = audio_feats[feat]
    for feat in categorical_features: 
        val = '_fulltrack_' + str(audio_feats[feat])
        feature[feat+val] = 1
    return feature


In [82]:
from collections import Counter
import librosa

def get_extensive_audio_features(sp, sample_trackid, pvalues= [10,25,50,75,90], perform_dsp = True, return_basic_features = False): 
    '''
    Get a single track and retrieve audio features that are in addition to the basic features.
    '''
    pitches = 'C C# D D# E F F# G G# A A# B'.split(' ')
    timbres = ['timbre_'+str(a) for a in range(12)]

    features = {'id': sample_trackid}
    
    if return_basic_features: 
        features.update(gen_basic_audio_features(sp, sample_trackid))
    
    sample_audio_analysis = sp.audio_analysis(sample_trackid)

    # processing a single track 
    num_sections = len(sample_audio_analysis['sections'])
    
    loudness = [sample_audio_analysis['sections'][a]['loudness'] for a in range(num_sections)]
    tempo = [sample_audio_analysis['sections'][a]['tempo'] for a in range(num_sections)]
    features.update(get_stats(numpy.array(loudness), 'loudness'))
    features.update(get_stats(numpy.array(tempo), 'tempo'))

    key = ['key_'+str(sample_audio_analysis['sections'][a]['key']) for a in range(num_sections)]
    mode = ['mode_'+str(sample_audio_analysis['sections'][a]['mode']) for a in range(num_sections)]
    key_counter = dict(Counter(key))
    _sum = sum(key_counter.values())
    key_counter = {a:round(b/_sum*100)/100. for a,b in key_counter.items()}
    mode_counter = dict(Counter(mode))
    _sum = sum(mode_counter.values())
    mode_counter = {a:round(b/_sum*100)/100. for a,b in mode_counter.items()}

    features.update(key_counter)
    features.update(mode_counter)


    # pitches and timbre
    num_segments = len(sample_audio_analysis['segments'])
    pitch_data = []
    timbre_data = []
    for a in range(num_segments): 
        pitch_data.append(sample_audio_analysis['segments'][a]['pitches'])
        timbre_data.append(sample_audio_analysis['segments'][a]['timbre'])
    pitch_df = pandas.DataFrame(data = numpy.array(pitch_data), columns = pitches)
    timbre_df = pandas.DataFrame(data = numpy.array(timbre_data), columns = timbres)

    for p in pitches: 
        features.update(get_stats(pitch_df[p].values, p, pvalues = [10,25,50,75,90]))
    for t in timbres: 
        features.update(get_stats(timbre_df[t].values, t, pvalues = [10,25,50,75,90]))

    if perform_dsp: 
        sample_track = sp.track(sample_trackid)
        audio_file = wget.download(sample_track['preview_url'])
        audio, sr  = librosa.load(audio_file)
        os.remove(audio_file)

        zero_crossings = librosa.feature.zero_crossing_rate(audio)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)

        features.update(get_mean_variance(zero_crossings, col_prefix='zero_crossings'))
        features.update(get_mean_variance(spectral_centroids, col_prefix='spectral_centroids'))
        features.update(get_mean_variance(rolloff, col_prefix='rolloff'))
        
    return features
    


In [83]:
def gen_extensive_audio_features(sp, trackids, pvalues= [10,25,50,75,90], perform_dsp = True, return_basic_features = False): 
    features_all_tracks = []
    for ind, t_id in enumerate(trackids): 
        print('processing track ', t_id, '{f} out of {e}'.format(f=ind, e=len(trackids)))
        features_all_tracks.append(get_extensive_audio_features(sp, t_id, pvalues= [10,25,50,75,90], perform_dsp = perform_dsp, return_basic_features = return_basic_features))

    df_audio_extra = pandas.DataFrame(features_all_tracks)
    return df_audio_extra

## Testing Features on a single track 

In [71]:
trackids = get_recommendations_tracks_from_genre(sp,['classical'], limit=5)
sample_trackid = trackids[5]

In [73]:
sample_trackid
sp.audio_features(sample_trackid)


[{'danceability': 0.251,
  'energy': 0.06,
  'key': 0,
  'loudness': -24.526,
  'mode': 1,
  'speechiness': 0.0431,
  'acousticness': 0.931,
  'instrumentalness': 0.809,
  'liveness': 0.16,
  'valence': 0.0729,
  'tempo': 135.354,
  'type': 'audio_features',
  'id': '7JUbQkbvdPnNqS8RdcfCtO',
  'uri': 'spotify:track:7JUbQkbvdPnNqS8RdcfCtO',
  'track_href': 'https://api.spotify.com/v1/tracks/7JUbQkbvdPnNqS8RdcfCtO',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7JUbQkbvdPnNqS8RdcfCtO',
  'duration_ms': 374547,
  'time_signature': 4}]

In [74]:
sp.audio_features(trackids)

[{'danceability': 0.422,
  'energy': 0.0461,
  'key': 10,
  'loudness': -26.781,
  'mode': 1,
  'speechiness': 0.04,
  'acousticness': 0.977,
  'instrumentalness': 0.839,
  'liveness': 0.0557,
  'valence': 0.337,
  'tempo': 118.922,
  'type': 'audio_features',
  'id': '11WQasJcGWgh3mvR1UiBcm',
  'uri': 'spotify:track:11WQasJcGWgh3mvR1UiBcm',
  'track_href': 'https://api.spotify.com/v1/tracks/11WQasJcGWgh3mvR1UiBcm',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/11WQasJcGWgh3mvR1UiBcm',
  'duration_ms': 189085,
  'time_signature': 4},
 {'danceability': 0.241,
  'energy': 0.099,
  'key': 9,
  'loudness': -26.281,
  'mode': 1,
  'speechiness': 0.0422,
  'acousticness': 0.92,
  'instrumentalness': 0.934,
  'liveness': 0.103,
  'valence': 0.0986,
  'tempo': 106.902,
  'type': 'audio_features',
  'id': '7hVsNjrvtfXZlzxW5vxsAi',
  'uri': 'spotify:track:7hVsNjrvtfXZlzxW5vxsAi',
  'track_href': 'https://api.spotify.com/v1/tracks/7hVsNjrvtfXZlzxW5vxsAi',
  'analysis_url': 'https:/

In [75]:
numerical_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness']
numerical_features += ['valence', 'tempo', 'duration_ms']
categorical_features = ['key', 'mode']
feature_df = get_audio_df(sp, trackids, numerical_features, categorical_features)
feature_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key_11,key_2,key_3,key_4,key_5,key_7,key_8,key_9,mode_1,id
0,0.422,0.0461,-26.781,0.0400,0.977,0.839,0.0557,0.3370,118.922,189085,...,0,0,0,0,0,0,0,0,1,11WQasJcGWgh3mvR1UiBcm
1,0.241,0.0990,-26.281,0.0422,0.920,0.934,0.1030,0.0986,106.902,227288,...,0,0,0,0,0,0,0,1,1,7hVsNjrvtfXZlzxW5vxsAi
2,0.444,0.1160,-23.172,0.0410,0.977,0.868,0.0862,0.5120,150.319,145613,...,0,0,0,1,0,0,0,0,0,06am46cX3Z6YlSsg0TyVHA
3,0.256,0.2460,-21.403,0.0490,0.901,0.297,0.5780,0.6390,62.102,134192,...,0,0,0,0,0,0,0,0,1,6yTPBtEggdlBgGEUL94sn1
4,0.290,0.0572,-27.498,0.0443,0.971,0.705,0.3530,0.1520,145.487,327973,...,0,0,0,1,0,0,0,0,0,2NodH0sMaxRFSWePxkuutP


In [76]:
gen_basic_audio_features(sp, sample_trackid)

{'danceability': 0.251,
 'energy': 0.06,
 'loudness': -24.526,
 'speechiness': 0.0431,
 'acousticness': 0.931,
 'instrumentalness': 0.809,
 'liveness': 0.16,
 'valence': 0.0729,
 'tempo': 135.354,
 'duration_ms': 374547,
 'key_0': 1,
 'mode_1': 1}

In [84]:
# extensive features for sample track 

get_extensive_audio_features(sp, sample_trackid, pvalues= [10,25,50,75,90], perform_dsp = False, return_basic_features=True)


{'id': '7JUbQkbvdPnNqS8RdcfCtO',
 'danceability': 0.251,
 'energy': 0.06,
 'loudness': -24.526,
 'speechiness': 0.0431,
 'acousticness': 0.931,
 'instrumentalness': 0.809,
 'liveness': 0.16,
 'valence': 0.0729,
 'tempo': 135.354,
 'duration_ms': 374547,
 'key_fulltrack_0': 1,
 'mode_fulltrack_1': 1,
 'loudness_0': -36.69,
 'loudness_25': -31.53,
 'loudness_50': -27.08,
 'loudness_75': -24.82,
 'loudness_100': -18.06,
 'tempo_0': 78.27,
 'tempo_25': 81.36,
 'tempo_50': 84.77,
 'tempo_75': 133.41,
 'tempo_100': 137.76,
 'key_4': 0.13,
 'key_0': 0.4,
 'key_9': 0.27,
 'key_1': 0.13,
 'key_2': 0.07,
 'mode_1': 0.93,
 'mode_0': 0.07,
 'C_10': 0.07,
 'C_25': 0.13,
 'C_50': 0.32,
 'C_75': 0.81,
 'C_90': 1.0,
 'C#_10': 0.07,
 'C#_25': 0.13,
 'C#_50': 0.24,
 'C#_75': 0.52,
 'C#_90': 1.0,
 'D_10': 0.05,
 'D_25': 0.09,
 'D_50': 0.19,
 'D_75': 0.42,
 'D_90': 0.84,
 'D#_10': 0.04,
 'D#_25': 0.07,
 'D#_50': 0.14,
 'D#_75': 0.26,
 'D#_90': 0.43,
 'E_10': 0.04,
 'E_25': 0.1,
 'E_50': 0.22,
 'E_75': 0.5

In [86]:
import time 
now = time.time()
testingdf= gen_extensive_audio_features(sp, trackids, pvalues= [10,25,50,75,90], perform_dsp = False, return_basic_features=True)
print(time.time()- now)

processing track  11WQasJcGWgh3mvR1UiBcm 0 out of 85
processing track  7hVsNjrvtfXZlzxW5vxsAi 1 out of 85
processing track  06am46cX3Z6YlSsg0TyVHA 2 out of 85
processing track  6yTPBtEggdlBgGEUL94sn1 3 out of 85
processing track  2NodH0sMaxRFSWePxkuutP 4 out of 85
processing track  7JUbQkbvdPnNqS8RdcfCtO 5 out of 85
processing track  460K7NeHx8ttnZa0Qqh4Vx 6 out of 85
processing track  1npEvrkd5GA8djI76r68Ay 7 out of 85
processing track  2bM4iYnqICtB3vAGplnz0d 8 out of 85
processing track  5jXB4p8pQu9Ahv10ibU8fG 9 out of 85
processing track  7KwH7xYAQ5rFja2GnJLIqD 10 out of 85
processing track  3eHV5kXt0OZmrUTrO8wzMX 11 out of 85
processing track  0W5hxw5mvLIVZu1QIw7IpP 12 out of 85
processing track  13rYowrPC2dk2Jhavquy1F 13 out of 85
processing track  7o5DCsdl4eIqusst2pVoKX 14 out of 85
processing track  6kHWawEbq5Vcyp8nSHhGjs 15 out of 85
processing track  2FSMEjWvRqmrdZbsqiW8ds 16 out of 85
processing track  4L9KpsmAbyhXOZZkSybfaU 17 out of 85
processing track  0CJuLeZpCdqrqoCMXWZw

In [87]:
testingdf.head()

,id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,key_1,key_fulltrack_2,key_3,key_fulltrack_3,key_fulltrack_7,key_fulltrack_5,key_fulltrack_11,key_6,key_fulltrack_1,key_fulltrack_8
0,11WQasJcGWgh3mvR1UiBcm,0.422,0.0461,-26.781,0.0400,0.977,0.839,0.0557,0.3370,118.922,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7hVsNjrvtfXZlzxW5vxsAi,0.241,0.0990,-26.281,0.0422,0.920,0.934,0.1030,0.0986,106.902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06am46cX3Z6YlSsg0TyVHA,0.444,0.1160,-23.172,0.0410,0.977,0.868,0.0862,0.5120,150.319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6yTPBtEggdlBgGEUL94sn1,0.256,0.2460,-21.403,0.0490,0.901,0.297,0.5780,0.6390,62.102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2NodH0sMaxRFSWePxkuutP,0.290,0.0572,-27.498,0.0443,0.971,0.705,0.3530,0.1520,145.487,...,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df_audio_extra = testingdf
df_audio_extra.columns.tolist()
print(df_audio_extra.dtypes)
print(feature_df.dtypes)

id              object
loudness_0     float64
loudness_25    float64
loudness_50    float64
loudness_75    float64
                ...   
key_11         float64
key_4          float64
key_5          float64
key_0          float64
key_9          float64
Length: 145, dtype: object


NameError: name 'feature_df' is not defined

In [89]:
## use this code in case we decide to return a dataframe for basic features
feature_df = get_audio_df(sp, trackids, numerical_features, categorical_features)
df_audio_extra = gen_extensive_audio_features(sp, trackids, pvalues= [10,25,50,75,90], perform_dsp = False)
joined_df = pandas.merge(feature_df, df_audio_extra, how='inner', on='id')
print(joined_df.columns.tolist())
joined_df.head()


processing track  11WQasJcGWgh3mvR1UiBcm 0 out of 85
processing track  7hVsNjrvtfXZlzxW5vxsAi 1 out of 85
processing track  06am46cX3Z6YlSsg0TyVHA 2 out of 85
processing track  6yTPBtEggdlBgGEUL94sn1 3 out of 85
processing track  2NodH0sMaxRFSWePxkuutP 4 out of 85
processing track  7JUbQkbvdPnNqS8RdcfCtO 5 out of 85
processing track  460K7NeHx8ttnZa0Qqh4Vx 6 out of 85
processing track  1npEvrkd5GA8djI76r68Ay 7 out of 85
processing track  2bM4iYnqICtB3vAGplnz0d 8 out of 85
processing track  5jXB4p8pQu9Ahv10ibU8fG 9 out of 85
processing track  7KwH7xYAQ5rFja2GnJLIqD 10 out of 85
processing track  3eHV5kXt0OZmrUTrO8wzMX 11 out of 85
processing track  0W5hxw5mvLIVZu1QIw7IpP 12 out of 85
processing track  13rYowrPC2dk2Jhavquy1F 13 out of 85
processing track  7o5DCsdl4eIqusst2pVoKX 14 out of 85
processing track  6kHWawEbq5Vcyp8nSHhGjs 15 out of 85
processing track  2FSMEjWvRqmrdZbsqiW8ds 16 out of 85
processing track  4L9KpsmAbyhXOZZkSybfaU 17 out of 85
processing track  0CJuLeZpCdqrqoCMXWZw

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key_2_y,key_7_y,key_11_y,key_4_y,key_8_y,key_0,mode_0,key_1_y,key_3_y,key_6
0,0.422,0.0461,-26.781,0.0400,0.977,0.839,0.0557,0.3370,118.922,189085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.241,0.0990,-26.281,0.0422,0.920,0.934,0.1030,0.0986,106.902,227288,...,0.08,0.25,0.17,0.08,0.08,0.08,0.42,NaN,NaN,NaN
2,0.444,0.1160,-23.172,0.0410,0.977,0.868,0.0862,0.5120,150.319,145613,...,NaN,0.33,NaN,0.67,NaN,NaN,0.50,NaN,NaN,NaN
3,0.256,0.2460,-21.403,0.0490,0.901,0.297,0.5780,0.6390,62.102,134192,...,0.38,NaN,NaN,NaN,NaN,0.25,0.12,NaN,NaN,NaN
4,0.290,0.0572,-27.498,0.0443,0.971,0.705,0.3530,0.1520,145.487,327973,...,0.19,0.12,NaN,0.19,NaN,0.12,0.25,0.06,NaN,NaN


In [90]:
## sanity 

print(len(feature_df), len(df_audio_extra), len(joined_df), len(trackids))

85 85 85 85
